In [87]:
import pandas as pd 

In [88]:
taxi_df = pd.read_parquet('Data/yellow_tripdata_2025-03.parquet')

In [89]:
taxi_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,1,2025-03-01 00:17:16,2025-03-01 00:25:52,1.0,0.90,1.0,N,140,236,1,7.9,3.50,0.5,2.60,0.0,1.0,15.50,2.5,0.0,0.00
1,1,2025-03-01 00:37:38,2025-03-01 00:43:51,1.0,0.60,1.0,N,140,262,1,6.5,3.50,0.5,2.30,0.0,1.0,13.80,2.5,0.0,0.00
2,2,2025-03-01 00:24:35,2025-03-01 00:39:49,1.0,1.94,1.0,N,161,68,1,14.9,1.00,0.5,5.16,0.0,1.0,25.81,2.5,0.0,0.75
3,2,2025-03-01 00:56:16,2025-03-01 01:01:35,2.0,0.95,1.0,N,231,13,1,7.2,1.00,0.5,2.59,0.0,1.0,15.54,2.5,0.0,0.75
4,1,2025-03-01 00:01:44,2025-03-01 00:10:00,1.0,1.50,1.0,N,163,236,1,8.6,4.25,0.5,2.85,0.0,1.0,17.20,2.5,0.0,0.75


In [90]:
taxi_df.shape

(4145257, 20)

In [91]:
taxi_df.isna().sum()

VendorID                      0
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count          916663
trip_distance                 0
RatecodeID               916663
store_and_fwd_flag       916663
PULocationID                  0
DOLocationID                  0
payment_type                  0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
congestion_surcharge     916663
Airport_fee              916663
cbd_congestion_fee            0
dtype: int64

In [92]:
taxi_df.describe(include='all').transpose()

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
VendorID,4145257.0,NaN,NaN,NaN,1.824944,1.0,2.0,2.0,2.0,7.0,0.549077
tpep_pickup_datetime,4145257,NaN,NaN,NaN,2025-03-16 13:03:28.198472,2007-12-05 18:45:00,2025-03-08 16:22:48,2025-03-16 03:02:13,2025-03-24 09:14:15,2025-04-01 00:00:17,NaN
tpep_dropoff_datetime,4145257,NaN,NaN,NaN,2025-03-16 13:19:27.549571,2007-12-05 19:02:00,2025-03-08 16:39:00,2025-03-16 03:15:59,2025-03-24 09:31:00,2025-04-03 14:07:50,NaN
passenger_count,3228594.0,NaN,NaN,NaN,1.29112,0.0,1.0,1.0,1.0,9.0,0.73175
trip_distance,4145257.0,NaN,NaN,NaN,6.5841,0.0,1.03,1.8,3.42,320136.29,626.407537
RatecodeID,3228594.0,NaN,NaN,NaN,2.423492,1.0,1.0,1.0,1.0,99.0,11.3601
store_and_fwd_flag,3228594,2,N,3221152,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PULocationID,4145257.0,NaN,NaN,NaN,161.765313,1.0,114.0,161.0,232.0,265.0,65.921996
DOLocationID,4145257.0,NaN,NaN,NaN,161.228734,1.0,107.0,162.0,233.0,265.0,70.132795
payment_type,4145257.0,NaN,NaN,NaN,0.956095,0.0,1.0,1.0,1.0,4.0,0.736272


In [93]:
taxi_df[taxi_df['total_amount']<0].shape

(68686, 20)

In [94]:
taxi_df[taxi_df['total_amount'] < 0].head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
112,2,2025-03-01 00:03:53,2025-03-01 00:11:31,1.0,1.66,1.0,N,114,68,4,-10.0,-1.0,-0.5,0.0,0.0,-1.0,-15.75,-2.5,0.0,-0.75
157,2,2025-03-01 00:46:20,2025-03-01 00:46:26,1.0,0.01,2.0,N,163,163,2,-70.0,0.0,-0.5,0.0,0.0,-1.0,-74.75,-2.5,0.0,-0.75
362,2,2025-03-01 00:07:57,2025-03-01 00:13:36,1.0,0.79,1.0,N,186,246,4,-7.2,-1.0,-0.5,0.0,0.0,-1.0,-12.95,-2.5,0.0,-0.75
423,2,2025-03-01 00:08:03,2025-03-01 00:33:51,2.0,4.59,1.0,N,144,170,4,-27.5,-1.0,-0.5,0.0,0.0,-1.0,-33.25,-2.5,0.0,-0.75
503,2,2025-03-01 00:28:29,2025-03-01 00:34:56,1.0,1.10,1.0,N,229,164,4,-8.6,-1.0,-0.5,0.0,0.0,-1.0,-14.35,-2.5,0.0,-0.75


In [95]:
taxi_df[taxi_df['total_amount'] < 0]['payment_type'].value_counts()

payment_type
4    42896
2    17032
3     8719
1       25
0       14
Name: count, dtype: int64

In [96]:
taxi_df[taxi_df['total_amount']==0].shape

(601, 20)

In [97]:
taxi_df[taxi_df['total_amount']==0].head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
14996,2,2025-03-01 04:27:20,2025-03-01 04:28:00,1.0,0.85,1.0,N,80,80,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41557,1,2025-03-01 13:14:34,2025-03-01 13:14:47,1.0,0.00,1.0,N,145,145,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51415,1,2025-03-01 15:35:38,2025-03-01 15:36:02,1.0,0.00,1.0,N,262,262,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58747,1,2025-03-01 16:17:50,2025-03-01 16:26:12,1.0,1.40,1.0,N,43,237,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86303,2,2025-03-01 19:06:10,2025-03-01 19:19:50,1.0,13.35,1.0,N,135,226,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [98]:
taxi_df[taxi_df['total_amount'] == 0]['payment_type'].value_counts()

payment_type
4    244
3    169
2     73
1     64
0     51
Name: count, dtype: int64

In [99]:
taxi_df = taxi_df[taxi_df['total_amount'] >= 0]

In [100]:
taxi_df['total_amount'].describe()

count    4.076571e+06
mean     2.718305e+01
std      3.121880e+01
min      0.000000e+00
25%      1.596000e+01
50%      2.105000e+01
75%      2.965000e+01
max      4.626944e+04
Name: total_amount, dtype: float64

In [106]:
taxi_df[taxi_df['total_amount']>250].shape

(1048, 20)

In [107]:
taxi_df[taxi_df['total_amount']>250].head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
16458,2,2025-03-01 06:23:52,2025-03-01 07:22:05,1.0,52.55,4.0,N,132,265,4,285.8,0.0,0.5,0.00,6.94,1.0,295.99,0.0,1.75,0.00
18476,2,2025-03-01 08:03:05,2025-03-01 08:53:10,2.0,42.89,4.0,N,132,265,1,233.3,0.0,0.0,20.00,6.94,1.0,261.24,0.0,0.00,0.00
19716,2,2025-03-01 08:07:48,2025-03-01 08:59:53,1.0,42.14,4.0,N,132,265,1,228.4,0.0,0.5,47.37,6.94,1.0,285.96,0.0,1.75,0.00
19993,2,2025-03-01 08:34:11,2025-03-01 09:43:13,1.0,42.06,4.0,N,132,265,1,229.1,0.0,0.0,56.00,21.00,1.0,309.60,0.0,1.75,0.75
20060,2,2025-03-01 08:20:32,2025-03-01 09:33:07,1.0,46.61,4.0,N,132,265,1,242.4,0.0,0.0,0.00,21.00,1.0,266.15,0.0,1.75,0.00


In [108]:
taxi_df = taxi_df[
    (taxi_df['total_amount'] > 0) &
    (taxi_df['total_amount'] <= 250)
]

In [109]:
taxi_df.shape

(4074922, 20)

In [110]:
taxi_df[taxi_df['fare_amount']<0].shape

(140446, 20)

In [111]:
taxi_df[taxi_df['fare_amount'] < 0].head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
3228594,2,2025-03-01 00:38:00,2025-03-01 00:47:00,NaN,1.69,NaN,None,68,249,0,-4.75,0.0,0.5,0.0,0.0,1.0,4.76,NaN,NaN,0.75
3228596,2,2025-03-01 00:27:50,2025-03-01 00:35:59,NaN,1.62,NaN,None,42,41,0,-1.50,0.0,0.5,0.0,0.0,1.0,2.00,NaN,NaN,0.00
3228600,2,2025-03-01 00:05:39,2025-03-01 00:09:26,NaN,0.80,NaN,None,234,90,0,-4.75,0.0,0.5,0.0,0.0,1.0,5.29,NaN,NaN,0.75
3228605,2,2025-03-01 00:04:04,2025-03-01 00:17:39,NaN,1.79,NaN,None,163,170,0,-4.75,0.0,0.5,0.0,0.0,1.0,2.97,NaN,NaN,0.75
3228607,2,2025-03-01 00:31:40,2025-03-01 00:40:36,NaN,2.23,NaN,None,140,75,0,-4.00,0.0,0.5,0.0,0.0,1.0,1.00,NaN,NaN,0.00


In [112]:
taxi_df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee',
       'cbd_congestion_fee'],
      dtype='object')

In [113]:
taxi_df = taxi_df[[
    'VendorID',
    'tpep_pickup_datetime',
    'tpep_dropoff_datetime',
    'trip_distance',
    'RatecodeID',
    'store_and_fwd_flag',
    'PULocationID',
    'DOLocationID',
    'payment_type',
    'total_amount'
]]

In [117]:
taxi_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,total_amount
0,1,2025-03-01 00:17:16,2025-03-01 00:25:52,0.90,1.0,N,140,236,1,15.50
1,1,2025-03-01 00:37:38,2025-03-01 00:43:51,0.60,1.0,N,140,262,1,13.80
2,2,2025-03-01 00:24:35,2025-03-01 00:39:49,1.94,1.0,N,161,68,1,25.81
3,2,2025-03-01 00:56:16,2025-03-01 01:01:35,0.95,1.0,N,231,13,1,15.54
4,1,2025-03-01 00:01:44,2025-03-01 00:10:00,1.50,1.0,N,163,236,1,17.20
